# Introduction

First we create a Pomegranate Bayesian Network using the protobuf interface and our utility package, bayesnet, that enables one to conveniently create a Bayesian network by hand, and in accord with common statistics and rules, without having to fill in every probability.  The same Baysian network can start with our hand entered rules, but can later learn from data.  Any amount of questions can be answered, including no questions, to get the probabilities .  The file the Bayesian net is written in is longevity_bayes.py.  Our utility package can then be used to query the network with different evidence of states.  This is convenient for tuning hand entered parameters.

## Longevity net
Here put the name of the network you have written with the protobuf and bayes utilities.  Our example is printed out.  We created 4 functions in our utility package: any_of, all_of, if_the_else, and avg, which can together be used to express almost any set of rules, documented elsewhere.  We also implemented a dependency rule with which one can enter statistics from Systematic Reviews and Meta analyses easily.  The last lines implement the creation of the description of the network,  bayesianNetwork , with the protobuf utilities, and then the creation of the Pomegranate network with the description.  

## Timing:  compiling, computation, explaning
It takes about 1 hour and 16 minutes to compile the file.  However, the user does not experience this delay, just the programmer.  Later this notebook you will see that the computation time, to get to the answer when it is running, is about 7 seconds.  Times to get an explanation depend on how many possible explanations we want to cover: for example, if there are 60 things to check, it takes 16 seconds.  We have made a way to retrieve the answers gradually over that time, if necessary.

## Variables to connect to , in order to fill in the health recommendations

1. diet:  "poor_diet".  upshot will not include socioeconomic status in recommendations and will recommend to increase or decrease things in diet, including calories
2. supplements:  "poor_diet_flag".  This shows area of deficiency that may need supplementation over and above diet
3. sleep hygiene:  "poor_sleep"
4. body weight: "obesity"
5. exercise: "lack_of_exercise"
6. lifestyle:   - "poor_diet_quantity"(water caffine sodium alcohol) + "psychological_disorders"(anxiety and marital status) + "smoking".  Just add to each other, do not rank with the same numbers to each.


## Variables to conect to, in order to fill in the hallmarks

1. "hallmark_1_genomic_instability"
2. "hallmark_2_telomere_attrition"
3. "hallmark_3_epigenetic_alterations"
4. "hallmark_4_loss_of_proteostasis"
5. "hallmark_5_deregulated_nutrient_sensing"
6. "hallmark_6_mitochondrial_dysfunction"
7. "hallmark_7_cellular_senescence"
8. "hallmark_8_stem_cell_exhaustion"
9. "hallmark_9_altered_intercellular_communication"
10. "hallmark_10_extracellular_matrix_dysfunction"


## Testing the Bayesian Network

Here we use our query util to enter patient states as evidence (evidence), and then see the probabilities of particular variables we are interested in (outvars), in this case, the output variables. The query function needs the compiled Pomegranate Bayesian network as well as the description of the network. In the following examples, we enter a particular patients state , one question at a time, to get a continual change in probability of all states.Adding more risk factors increases risk more. We test the bayesian net through the internal query, as it is done for efficiency in the app. Note the internal query sends back all states not just the requested ones. We also use query, so that we can see the results clearly in this notbook. If most persons are of low socioeconomic status to begin with (65% in Nhanes data), then that would explain why the values only went up slightly even though the relative risk of poor diet given low socioeconomic status is 2.66.

## Testing explanations
The explain function will tell you which nodes, whether inner or leaf, when changed, change the output variable.  A variable is on the reverse explain list if we want to know what changes to other variables will increase it.  Otherwise we look to 
changes that decrease the variable to be explained.  One may select which explanations are relevant for their purposes, as this is just a measure of dependence. The point is to find out which variables help a situation, where the output variables are the goals.  At default values,  we want to find out how to decrease an output_variables probability, when we move a variable from where it is to one value to the right.  Our present rules, in longevity_bayes.py, are designed so that one value to the right is an improvement.  The explanation output tells you the change in the probability of the output variable when this happens.  Those explanations with the greatest values tell the variables that when you improve them decrease the probability of the output variable the most, and are thus the most helpful in decreasing the chance of a bad thing.  However, some variables could possibly need to additionally be on the reverse_explain_list because they are not bad things, they are good things, so as a goal, you would want to know how to increase their probability not decrease it.  So, to find an explanation for why an indicator is so good, you have to enter all the nodes of the net into reverse_evidence that were not in reverse_evidence when explainng why an indicator is doing poorly.

Although we need the explain function for complex rule sets in which the direction of the rules  changes at times because there are different and multiple goals, we have made two wrapper functions that only apply for rulesets in which all the rules are in the same direction, for example the covid ruleset and the longevity ruleset.  One wrapper is explain_why_bad, which takes on the default of the explain function, and has no reverse_explain_list or reverse_evidence.  The other is explain_why_good, explained below.  explain_why_good (and explain using reverse evidence and explain vars only works when the result is actually good, and explain_why_bad (and the explain default) only works when a result is actually bad.  Below we show only the explain why good and explain why bad functions for explanations, because the longevity_bayes.py file only has rules that are expressed as negative states.

## Explanations

To pull out explanations, the algorithm has changed as a result of the new less computationally complex algorithm.  First, find the majority explanation value for an output variable, and then get rid of all candidate explanations that have that value or below. Then, of the remaining, use a threshold of 0.1 and above to include an explanation.  Then you can rank them by the magnitude, to find the stronger explanations, perhaps to show those nearer to the top.  Below, for example, we have preinted out the why bad, and then the why good. Typically if the results are good, and you ask why they are bad, you will not get an answer and visa versa.  We see that cancer is an explaination fo rwhy hallmark 7 and 9 are bad, and frailty explains 8 and 5. 

## Realistic survey answers

The above example is simple for understanding's sake, it just has 2 inputs, one of which is in the questions (cancer) and one which is not, but is affected by the questions that are inputs (frailty).  So next we will have the input that is more like the answers to the surveys, and see if they still produce correct results

# Requirements

# Imports

In [1]:
import qpsolvers
from os.path import exists
import pickle

import sn_bayes
from sn_bayes import longevity_bayes
from sn_bayes.utils import complexity_check
from sn_bayes.utils import get_var_positions
from sn_bayes.utils import get_var_val_positions
from sn_bayes.utils import make_tree
from sn_bayes.utils import bayesInitialize
from sn_bayes.utils import query
from sn_bayes.utils import internal_query
from sn_bayes.utils import internal_query
from sn_bayes.utils import explain_why_bad
from sn_bayes.utils import explain_why_good
from sn_bayes.utils import create_query

import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import Query
import sn_service.service_spec.bayesian_pb2_grpc as grpc_bayes_grpc
import sn_service.service_spec.bayesian_pb2
from sn_service.service_spec.bayesian_pb2 import BayesianNetworkQuery
from sn_service.service_spec.bayesian_pb2 import QueryId
from sn_service.service_spec.bayesian_pb2 import Id

import grpc
import pandas as pd
import networkx as nx
import time
import re

import matplotlib.pyplot as plt

# Pipeline

## Longevity net

In [2]:
if exists("bayesianNetwork.pkl"):
        infile = open("bayesianNetwork.pkl",'rb')
        bayesianNetwork = pickle.load(infile)
        infile.close()
else:
        %time bayesianNetwork,outstr = longevity_bayes.longevity_bayes()
        print (outstr)
        outfile = open("bayesianNetwork.pkl",'wb')
        pickle.dump(bayesianNetwork,outfile)
        outfile.close()

In [3]:
path = './sn_bayes'
text_file = open(f'{path}/longevity_bayes.py')
#text_file = open('./sn_bayes/longevity_bayes_short.py')
file_content = text_file.read()
#print(file_content)
text_file.close()

## Timing:  compiling, computation, explaning

In [4]:
print(qpsolvers.available_solvers)

['clarabel', 'daqp', 'ecos', 'osqp', 'scs']


In [5]:
#
complexity_check(bayesianNetwork)
#
var_positions = get_var_positions(bayesianNetwork)
#
var_val_positions = get_var_val_positions(bayesianNetwork)

In [6]:
# We create a spreadsheet of the above for reference
rows_list = []
outname = "varvals.csv"
for var, valdict in var_val_positions.items():
    rowdict = {} 
    rows_list.append(rowdict)
    varstr= var +"("
    for val, pos in valdict.items():
        varstr += val
        varstr+= ","
            
    varstr=varstr[:-1]+")"
    rowdict["variable"] = varstr        
#
df = pd.DataFrame(rows_list)      
# df.to_csv(outname, index = False)

## Variables to connect to , in order to fill in the health recommendations

## Variables to conect to, in order to fill in the hallmarks

In [7]:
'''Here we create a spreadsheet that makes a Cartesian product of desired outputs with 
the remainder of the variables. This can be used to, for example, fill in a treatment 
recommendation for variables that are returned as causing the output in the explanation module'''
#cols = ["leaves","internal","output"]
desired_output =["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
rows_list = []
outname = "upshot.csv"
#put leaves in the first column, then internal vars along with the vars that feed them in the second.  
#and do a cartesian product with the outvars that are also internal vars (should be all of them)
for output in desired_output:
    for dist in bayesianNetwork.discreteDistributions:
        #print(dist.name)
        rowdict = {} 
        rowdict["leaves"]= dist.name
        rowdict["internal"]=""
        rowdict["output"]= output
        rows_list.append(rowdict)
    for table in bayesianNetwork.conditionalProbabilityTables:
        #print ("table: {}".format(table.name))
        rowdict = {}
        rowdict["leaves"]=""
        internal_str = table.name + " ("
        for var in table.randomVariables:
            #print(var.name)
            internal_str += var.name
            internal_str+= ","         
        internal_str=internal_str[:-1]+")"
        rowdict["internal"] = internal_str
        rowdict["output"]= output
        rows_list.append(rowdict)
for dist in bayesianNetwork.discreteDistributions:
    #print(dist.name)
    rowdict = {} 
    rowdict["leaves"]= dist.name
    rowdict["internal"]=""
    rowdict["output"]= ""
    rows_list.append(rowdict)
for table in bayesianNetwork.conditionalProbabilityTables:
    #print ("table: {}".format(table.name))
    rowdict = {}
    rowdict["leaves"]=""
    internal_str = table.name + " ("
    for var in table.randomVariables:
        #print(var.name)
        internal_str += var.name
        internal_str+= ","         
    internal_str=internal_str[:-1]+")"
    rowdict["internal"] = internal_str
    rowdict["output"]= ""
    rows_list.append(rowdict) 
#
df = pd.DataFrame(rows_list)      
# df.to_csv(outname, index = False)

In [8]:
# Another csv that uses a utility function to express the net in  a tree form, in a data frame and then prints it out
df = make_tree(bayesianNetwork)
# df.to_csv("tree.csv" , index = False)

In [9]:
G = nx.DiGraph()
regex = re.compile(r'^([a-z_]*)\(.*')
for index, row in df.iterrows():
    for col in df.columns:
        if row[col] is not None:
            node = row[col][:-1] if col == "level0" else row[col]
            G.add_node(node)           
for node1 in G.nodes:
    for node2 in G.nodes:
        match = regex.match(node1)
        if (node1+',' in node2) or (match is not None and match.group(1)+"," in node2): 
            G.add_edge(node1,node2)
            if match is not None and match.group(1)+"," in node2:
                print (node1)
                print(node2)        

angina(age)
reported_heart_symptoms(daily_asprin_prescription,daily_aspirin_compliance,angina,chest_pain,blood_relative_heart_attack_before_age_50)
daily_asprin_prescription(angina)
reported_heart_symptoms(daily_asprin_prescription,daily_aspirin_compliance,angina,chest_pain,blood_relative_heart_attack_before_age_50)
lack_of_exercise(lack_of_exercise_signal,lack_of_exercise_reported)
sleep_anomaly(lack_of_exercise,oxygen_anomaly)
obesity(bmi,obesity_factors)
oxygen_anomaly(obesity,heart_rate_anomaly)
obesity(bmi,obesity_factors)
inflammation_from_illness(blood_illness_indicators,obesity,asthma_attack_last_year,fatiqued_in_2_weeks)
obesity(bmi,obesity_factors)
cardiovascular_disease_from_behavior(age,obesity,daily_time_sitting,workday_sleep_hours_per_night,daily_fruit_vegetable_servings)
poor_sleep(poor_sleep_schedule,sleep_anomaly)
inflammation_from_behavior(uv_exposure,poor_oral_health,toxin_labs,poor_sleep,lack_of_exercise)
poor_sleep(poor_sleep_schedule,sleep_anomaly)
hallmark_3_epig

poor_diet_supplements(dietary_lycopene,dietary_lutein,dietary_selenium,dietary_choline)
poor_diet_quantity(poor_diet_substances,poor_diet_supplements,poor_diet_basics)
diet_vitamin_a(dietary_retinol,dietary_alpha_carotene,dietary_beta_carotene)
poor_diet_vitamins_and_minerals(poor_diet_minerals,poor_diet_vitamin_a,poor_diet_vitamin_b,poor_diet_vitamin_other,vitamin_and_mineral_labs)
diet_vitamin_a(dietary_retinol,dietary_alpha_carotene,dietary_beta_carotene)
diet_vitamins_and_minerals(diet_minerals,diet_vitamin_a,diet_vitamin_b,diet_vitamin_other)
diet_vitamin_other(dietary_vitamin_C,dietary_alpha_tocopherol,dietary_calcium)
poor_diet_vitamins_and_minerals(poor_diet_minerals,poor_diet_vitamin_a,poor_diet_vitamin_b,poor_diet_vitamin_other,vitamin_and_mineral_labs)
diet_fats(dietary_epa,dietary_pfa,dietary_fat,dietary_cholesterol)
poor_diet_food(poor_diet_fats,poor_diet_vitamins_and_minerals)
diet_fats(dietary_epa,dietary_pfa,dietary_fat,dietary_cholesterol)
diet_quality_food(how_healthy

In [10]:
# Pomegranate net we just made with the description. Compile the net so we can compute probabilities with it.
longevity = bayesInitialize(bayesianNetwork)
#
longevity.bake()
# Calls a Pomegranate routine that shows the computed probabilities of every variable. 
longevity.predict_proba({}) # We have our own utility that pulls out particular variables.

array([{
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "fake_variable_1" : 0.04999999717381816,
                   "fake_variable_2" : 0.25000005288880306,
                   "fake_variable_3" : 0.2999997861930073,
                   "fake_variable_4" : 0.200000162748353,
                   "fake_variable_5" : 0.20000000099601856
               }
           ],
           "frozen" : false
       }                                                   ,
       {
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "elderly" : 0.04996686518206726,
                   "adult" : 0.24996741459510588,
                   "young_adult" : 0.300066994693074,
                   "teen" : 0.19999936444026953,
                   "child" : 0.19999936108948324
    

## Testing the Bayesian Network

In [11]:
# Testing the Bayesian Network
evidence = {}
outvars1 = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results1 = query(longevity,bayesianNetwork,evidence,outvars1)

In [12]:
#
evidence["low_socioeconomic_status"] = "low_socioeconomic_status"
results2 = query(longevity,bayesianNetwork,evidence,outvars1)

In [13]:
# Tests of increasingly poor patient state, showing gradual change of hallmarks
evidence = {}
%time internal_query_result1 =  internal_query(longevity,bayesianNetwork,evidence)

Wall time: 3.78 s


In [14]:
#
evidence["low_socioeconomic_status"] = "low_socioeconomic_status"
internal_query_result2 =  internal_query(longevity,bayesianNetwork,evidence)

In [15]:
#
evidence = {}
evidence["frailty"] = "frailty"
internal_query_result3 =  internal_query(longevity,bayesianNetwork,evidence)

In [16]:
#
evidence["cancer"] = "cancer"
internal_query_result4 =  internal_query(longevity,bayesianNetwork,evidence)
# we show some of the same results as in the last cell for clarity
results3 = query(longevity,bayesianNetwork,evidence,outvars1)

## Testing Explanations

In [17]:
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
# include_list = ["heart_rate_variability_anomaly","oxygen_anomaly","lack_of_exercise","uv_exposure",
#                 "blood_age_indicators","poor_sleep","inflammation_from_behavior","poor_diet","smoking",
#                 "frailty_signals","cancer","frailty","liver_disorders","cardiovascular_disease",
#                 "inactivated_sirtuins","inflammation","sarcopenia","comorbidities","general_aging_signs",
#                 "blood_metabolism_disorder_indicators","obesity","hypertension","diabetes","metabolic_disease",
#                 "gender"]
include_list = ["age_related_macular_degeneration","alzheimers_disease","parkinsons_disease",
                "anxious_how_often","daily_time_sitting","daily_walk_or_bicycle_travel","dietary_fiber",
                "eat_out_how_many_times_per_week","five_days_smoke_cigarettes","gender","heart_rate_anomaly",
                "marital_status","step_variability_anomaly","steps_anomaly","ten_minutes_daily_workout",
                "workout_anomaly","blood_age_indicators","c_reactive_protein","cancer",
                "cardiovascular_disease","chronic_conditions","comorbidities","diabetes","diet_quality",
                "frailty","frailty_signals","frailty_signs","heart_rate_variability_anomaly","hypertension",
                "inactivated_sirtuins","inflammation","inflammation_from_behavior","inflammation_from_diet",
                "kidney_disease","lack_of_exercise","lung_or_kidney_or_liver_disease","metabolic_disease",
                "oxygen_anomaly","poor_diet","poor_diet_flag","poor_oral_health","poor_sleep",
                "psychological_disorders","sarcopenia","sleep_anomaly","smoking","uv_exposure",
                "liver_disorders","heart_disorder_indicators","inflammation_from_illness",
                "poor_diet_quantity","general_aging_signs","lack_of_exercise_reported","diet_basics",
                "diet_fats","diet_quality_food","diet_quantity","diet_vitamins_and_minerals",
                "blood_metabolism_disorder_indicators","obesity","poor_sleep_schedule"]
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time. 
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
# for check_if_explains in include_list:
#     checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result,include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

start timing...
Explanations took 15.882257199999998 seconds


## Explanations

In [18]:
evidence = {}
evidence1 = evidence.copy()
evidence1['a1c'] =  'a1c_diabetes_5.7_and_above'
%time internal_query_result5 =  internal_query(longevity,bayesianNetwork,evidence1)

Wall time: 4.06 s


## Realistic survey answers

In [19]:
evidence = {"age":"adult",
"daily_aspirin_compliance":"daily_aspirin_compliance_yes",
"high_blood_pressure_patient_prescription":"high_blood_pressure_patient_prescription_no",
"high_blood_pressure_medication_compliance":"high_blood_pressure_medication_compliance_yes",
"diastolic":"diastolic_80_to_89_high_stage_1",
"systolic":"systolic_less_than_120_normal",
"sun_exposure_half_hour":"sun_exposure_half_hour_mild_burn_tanning",
"time_spent_outdoors":"time_spent_outdoors_none",
"sun_exposure_use_sunscreen":"sun_exposure_use_sunscreen_sometimes",
"sunburn_last_year":"sunburn_last_year_yes",
"weight_at_25":"weight_at_25_above_125.00_to_150.00_and_below",
"bmi":"bmi_over_40_high_risk",
"dietary_energy":"dietary_energy_above_2612.00_to_3939.00_and_below",
"shortness_of_breath_exertion":"shortness_of_breath_exertion_yes",
"daily_walk_or_bicycle_travel":"daily_walk_or_bicycle_travel_less_than_30_minutes",
"daily_time_sitting":"daily_time_sitting_12_or_more_hours",
"physical_work":"physical_work_no",
"ten_minutes_daily_workout":"ten_minutes_daily_workout_no",
"heart_rate_anomaly":"heart_rate_anomaly",
"steps_anomaly":"steps_anomaly",
"workout_anomaly":"workout_anomaly",
"walking_speed_anomaly":"walking_speed_anomaly",
"walking_speed_variability_anomaly":"walking_speed_variability_anomaly",
"stride_variability_anomaly":"no_stride_variability_anomaly",
"step_asymmetry_anomaly":"no_step_asymmetry_anomaly",
"double_support_stepping_anomaly":"double_support_stepping__anomaly",
"step_variability_anomaly":"step_variability_anomaly",
"anxious_how_often":"anxious_daily",
"marital_status":"divorced",
"asthma_attack_last_year":"asthma_attack_last_year_no",
"liver_condition":"liver_condition_yes",
"fatiqued_in_2_weeks":"fatiqued_in_2_weeks_nearly_everyday",
"osteoporosis":"osteoporosis_no",
"artificial_joints":"artificial_joints_no",
"work_limiting_problem":"work_limiting_problem_yes",
"ten_stairs_difficulty":"ten_stairs_difficulty_some",
"chore_difficulty":"chore_difficulty_some",
"walking_difficulty":"walking_difficulty_much",
"stand_from_sit_difficulty":"stand_from_sit_difficulty_much",
"two_hour_standing_difficulty":"two_hour_standing_difficulty_much",
"two_hour_sitting_difficulty":"two_hour_sitting_difficulty_none",
"social_activity_difficulty":"social_activity_difficulty_much",
"urine_leakage_bother":"urine_leakage_bother_somewhat",
"hearing_difficulty_how_often":"hearing_difficulty_usually",
"hearing_frustration_how_often":"hearing_frustration_usually",
"wear_hearing_aid":"wear_hearing_aid_no",
"loud_noise_job":"loud_noise_job_no",
"loud_music":"loud_music_yes",
"second_hand_smoke":"second_hand_smoke_no",
"five_days_smoke_cigarettes":"five_days_smoke_cigarettes_no",
"ever_vaped":"ever_vaped_no",
"gender":"female",
"work_schedule":"work_schedule_evenings_or_nights",
"workday_sleep_hours_per_night":"workday_sleep_under_5",
"weekend_sleep_hours_per_night":"weekend_sleep_5.0_to_6.5",
"fish_last_month":"fish_last_month_no",
"tuna_eaten_times_per_month":"tuna_eaten_per_month_0_or_1",
"eat_out_how_many_times_per_week":"eat_out_more_than_7_times_per_week",
"how_healthy_diet":"how_healthy_diet_fair",
"alcohol_consumption_last_year":"alcohol_never",
"race":"race_white",
"education_level":"education_level_at_least_some_college",
"have_health_insurance":"have_health_insurance_yes",
"prescription_health_insurance":"prescription_health_insurance_yes",
"how_many_times_saw_doctor_last_year":"saw_doctor_1_or_2_times_last_year",
"how_long_since_saw_doctor":"saw_doctor_less_than_6_months_ago",
"what_is_your_annual_household_income":"over_100K",
"employment":"employment_working",
"age_at_first_child":"age_at_first_child_20_through_24",
"last_dentist_visit":"last_dentist_visit_6_months_or_less",
"dental_bone_loss":"dental_bone_loss_no",
"oral_cancer_exam":"oral_cancer_exam_no",
"lung_disease":"no_lung_disease",
"immunocompromised":"not_immunocompromised",
"angina":"angina_no",
"daily_asprin_prescription":"daily_asprin_prescription_no",
"blood_relative_heart_attack_before_age_50":"blood_relative_heart_attack_before_50_no",
"chest_pain":"chest_pain_no",
"greatest_weight":"greatest_weight_above_230.00",
"weight":"weight_above_92.60",
"height":"no_height_above_173.40",
"lack_of_exercise_reported":"lack_of_exercise_reported",
"salt_how_often":"salt_very_often",
"times_brush_teeth_daily":"no_times_brush_teeth_daily_1_or_less",
"floss_days_per_week":"no_floss_days_per_week_3_or_less",
"teeth_health":"teeth_health_poor",
"gum_disease":"gum_disease_yes",
"mouth_pain_last_year":"mouth_pain_last_year_fairly_often",
"sleep_anomaly":"sleep_anomaly"}

In [20]:
%time internal_query_result6 =  internal_query(longevity,bayesianNetwork,evidence)

Wall time: 3.57 s


In [21]:
# we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

start timing...
Explanations took 9.508365800000007 seconds


In [22]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'daily_walk_or_bicycle_travel': 0.5695114678772452, 'daily_time_sitting': 0.5695114678772453, 'ten_minutes_daily_workout': 0.5695114678772452, 'heart_rate_anomaly': 0.5695114683166467, 'steps_anomaly': 0.5695114643620338, 'workout_anomaly': 0.5695114678772452, 'step_variability_anomaly': 0.5695114678772452, 'anxious_how_often': 0.5695114678772452, 'marital_status': 0.5695114678772452, 'eat_out_how_many_times_per_week': 0.5695114678772452, 'lack_of_exercise_reported': 0.11865359750702498, 'sleep_anomaly': 0.438527497323068, 'uv_exposure': 0.5695114685363474, 'diet_vitamins_and_minerals': 0.5695114678772452, 'diet_fats': 0.5695114678772452, 'diet_basics': 0.5695114678772452, 'diet_quality_food': 0.5695114678772452, 'diet_quantity': 0.5695114678772452, 'diet_quality': 0.5695114678772452, 'inactivated_sirtuins': 0.5695114678772452, 'lung_or_kidney_or_liver_disease': 0.5695114678772452, 'chronic_conditions': 0.5695114678772452, 'comorbidities': 0.5695114678772452, 'fra

## One by One Survey Example (Apple Health + Apple Watch)

In order to sync the app with the notebook, do not turn on signals when testing in the following cells.  Signals are tested at the bottom of the notebook

In [23]:
include_list = ["dietary_energy", "bmi", "lack_of_exercise","anxious_how_often","daily_time_sitting",
                "daily_walk_or_bicycle_travel","dietary_fiber","eat_out_how_many_times_per_week",
                "five_days_smoke_cigarettes","gender","heart_rate_anomaly","marital_status",
                "step_variability_anomaly","steps_anomaly","ten_minutes_daily_workout","workout_anomaly",
                "blood_age_indicators","c_reactive_protein","cancer","cancer_related","cardiovascular_disease",
                "chronic_conditions","comorbidities","diabetes","diet_quality","frailty","frailty_signals",
                "frailty_signs","heart_rate_variability_anomaly","hypertension","inactivated_sirtuins",
                "inflammation","inflammation_from_behavior","inflammation_from_diet","kidney_disease",
                "lack_of_exercise","lung_or_kidney_or_liver_disease","metabolic_disease","oxygen_anomaly",
                "poor_diet","poor_diet_flag","poor_oral_health","poor_sleep","psychological_disorders",
                "sarcopenia","sleep_anomaly","smoking","uv_exposure","liver_disorders","heart_disorder_indicators",
                "inflammation_from_illness","poor_diet_quantity","general_aging_signs","lack_of_exercise_reported",
                "diet_basics","diet_fats","diet_quality_food","diet_quantity","diet_vitamins_and_minerals",
                "blood_metabolism_disorder_indicators","obesity","poor_sleep_schedule"]
#first, biometric survey                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
evidence = {"age": "adult",
        "bmi": "bmi_over_40_high_risk",
        "cancer": "no_cancer",
        "gender": "female",
        "height": "no_height_above_173.40",
        "weight": "weight_above_92.60",
        "diabetes": "no_diabetes",
        "systolic": "systolic_130_to_139_high_stage_1",
        "diastolic": "diastolic_80_to_89_high_stage_1",
        "hypertension": "no_hypertension",
        "lung_disease": "no_lung_disease",
        "kidney_disease": "no_kidney_disease",
        "immunocompromised": "not_immunocompromised",
        "alzheimers_disease": "no_alzheimers_disease",
        "parkinsons_disease": "no_parkinsons_disease",
        "resting_heart_rate": "no_resting_heart_rate_very_high_91_and_above",
        "cardiovascular_disease": "no_cardiovascular_disease",
        "psychological_disorders": "no_psychological_disorders",
        "age_related_macular_degeneration": "no_age_related_macular_degeneration"}          
#
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)

Wall time: 5.67 s


In [24]:
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

start timing...
Explanations took 7.188339899999988 seconds


In [25]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'bmi': 0.2127478043739297, 'obesity': 0.2127478043739297, 'diet_fats': 0.2127478043739297, 'diet_quality_food': 0.2127478043739297, 'diet_quantity': 0.2127478043739297, 'diet_quality': 0.2127478043739297, 'inactivated_sirtuins': 0.2127478043739297}, 'obesity': {'bmi': 0.6155373925543947, 'diet_fats': -0.009812653609235777, 'diet_quality_food': -0.009812653609235777, 'diet_quantity': -0.009812653609235777, 'diet_quality': -0.009812653609235777, 'inactivated_sirtuins': -0.009812653609235777}, 'smoking': {'bmi': 0.23925414734464123, 'obesity': 0.23925414734464123, 'diet_fats': 0.23925414734464123, 'diet_quality_food': 0.23925414734464123, 'diet_quantity': 0.23925414734464123, 'diet_quality': 0.23925414734464123, 'inactivated_sirtuins': 0.23925414734464123}, 'poor_sleep': {'bmi': 0.5106373051398498, 'obesity': 0.5106373051398498, 'diet_fats': 0.5106373051398498, 'diet_quality_food': 0.5106373051398498, 'diet_quantity': 0.5106373051398498, 'diet_quality': 0.51063730513

In [26]:
#add daily checkin
evidence.update(
    {
"workday_sleep_hours_per_night":"workday_sleep_under_5",
"water_drank_yesterday":"water_drank_yesterday_above_870.00_to_1633.50_and_below",
"dietary_energy":"dietary_energy_above_2612.00_to_3939.00_and_below" ,
"daily_walk_or_bicycle_travel":"daily_walk_or_bicycle_travel_less_than_30_minutes",
"daily_time_sitting":"daily_time_sitting_12_or_more_hours" ,         
"ten_minutes_daily_workout":"ten_minutes_daily_workout_no"
    }
)
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)

Wall time: 5.58 s


In [27]:
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

start timing...
Explanations took 8.160510400000007 seconds


In [28]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'bmi': -0.03265710753668616, 'dietary_energy': -0.03265711863949594, 'daily_walk_or_bicycle_travel': -0.03265710753668616, 'daily_time_sitting': -0.03265710753668605, 'ten_minutes_daily_workout': -0.03265710753668616, 'lack_of_exercise_reported': -0.2713722202439166, 'obesity': -0.03265710753668616, 'c_reactive_protein': -0.03265710753668616, 'poor_sleep_schedule': -0.03265710753668616, 'poor_sleep': -0.03265710614883499, 'diet_fats': -0.03265710753668616, 'diet_basics': -0.03265710753668616, 'diet_quality_food': -0.03265710753668616, 'diet_quantity': -0.03265710753668616, 'diet_quality': -0.03265710753668616, 'inactivated_sirtuins': -0.03265710753668616}, 'obesity': {'bmi': 0.6626387685798735, 'dietary_energy': 0.037288720698533506, 'daily_walk_or_bicycle_travel': 0.037288720698533506, 'daily_time_sitting': 0.03728871612991047, 'ten_minutes_daily_workout': 0.037288720698533506, 'lack_of_exercise_reported': 0.037288720698533506, 'c_reactive_protein': 0.03728872069

In [29]:
#add demographic survey
evidence.update(
    {
"race":"race_white",
"marital_status":"divorced",
"education_level":"education_level_at_least_some_college" ,
"what_is_your_annual_household_income":"over_100K",
"employment":"employment_working" ,       
"work_schedule":"work_schedule_variable",
"have_health_insurance":"have_health_insurance_yes",
"prescription_health_insurance":"prescription_health_insurance_yes"
    }
)
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

Wall time: 5.45 s
start timing...
Explanations took 8.364259099999998 seconds


In [30]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'bmi': -0.03297748089115504, 'dietary_energy': -0.032977491993964825, 'daily_walk_or_bicycle_travel': -0.03297748089115504, 'daily_time_sitting': -0.03297748089115493, 'ten_minutes_daily_workout': -0.03297748089115504, 'marital_status': -0.03297748089115504, 'lack_of_exercise_reported': -0.2716925935983855, 'obesity': -0.03297748089115504, 'c_reactive_protein': -0.03297748089115504, 'poor_sleep_schedule': -0.03297748089115504, 'poor_sleep': -0.032977479503303875, 'diet_fats': -0.03297748089115504, 'diet_basics': -0.03297748089115504, 'diet_quality_food': -0.03297748089115504, 'diet_quantity': -0.03297748089115504, 'diet_quality': -0.03297748089115504, 'inactivated_sirtuins': -0.03297748089115504}, 'obesity': {'bmi': 0.662676427872286, 'dietary_energy': 0.03732637999094601, 'daily_walk_or_bicycle_travel': 0.03732637999094601, 'daily_time_sitting': 0.03732637542232298, 'ten_minutes_daily_workout': 0.03732637999094601, 'marital_status': 0.03732637999094601, 'lack_of_

In [31]:
#add demographic survey
evidence.update(
    {
"race":"race_white",
"marital_status":"divorced",
"education_level":"education_level_at_least_some_college" ,
"what_is_your_annual_household_income":"over_100K",
"employment":"employment_working" ,       
"work_schedule":"work_schedule_variable",
"have_health_insurance":"have_health_insurance_yes",
"prescription_health_insurance":"prescription_health_insurance_yes"
    }
)
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

Wall time: 5.46 s
start timing...
Explanations took 8.318792700000017 seconds


In [32]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'bmi': -0.03297748089115504, 'dietary_energy': -0.032977491993964825, 'daily_walk_or_bicycle_travel': -0.03297748089115504, 'daily_time_sitting': -0.03297748089115493, 'ten_minutes_daily_workout': -0.03297748089115504, 'marital_status': -0.03297748089115504, 'lack_of_exercise_reported': -0.2716925935983855, 'obesity': -0.03297748089115504, 'c_reactive_protein': -0.03297748089115504, 'poor_sleep_schedule': -0.03297748089115504, 'poor_sleep': -0.032977479503303875, 'diet_fats': -0.03297748089115504, 'diet_basics': -0.03297748089115504, 'diet_quality_food': -0.03297748089115504, 'diet_quantity': -0.03297748089115504, 'diet_quality': -0.03297748089115504, 'inactivated_sirtuins': -0.03297748089115504}, 'obesity': {'bmi': 0.662676427872286, 'dietary_energy': 0.03732637999094601, 'daily_walk_or_bicycle_travel': 0.03732637999094601, 'daily_time_sitting': 0.03732637542232298, 'ten_minutes_daily_workout': 0.03732637999094601, 'marital_status': 0.03732637999094601, 'lack_of_

In [33]:
#add general medical history survey
evidence.update(
    {
        "how_long_since_saw_doctor":"saw_doctor_6_to_12_months_ago",
        "how_many_times_saw_doctor_last_year":"saw_doctor_1_or_2_times_last_year",
        "birth_weight":"birth_weight_7_to_8_lbs",
        "greatest_weight":"greatest_weight_above_230.00",
        "weight_at_25":"weight_at_25_above_125.00_to_150.00_and_below",
        "daily_asprin_prescription":"daily_asprin_prescription_no",
        "daily_aspirin_compliance":"daily_aspirin_compliance_no",
        "shortness_of_breath_exertion":"shortness_of_breath_exertion_yes",
        "work_limiting_problem":"work_limiting_problem_no",
        "anxious_how_often":"anxious_daily",
        "fatiqued_in_2_weeks":"fatiqued_in_2_weeks_more_than_half_the_days",
        "chest_pain":"chest_pain_no",
        "liver_condition":"liver_condition_yes",
        "osteoporosis":"osteoporosis_no",
        "asthma_attack_last_year":"asthma_attack_last_year_no",
        "high_blood_pressure_patient_prescription":"high_blood_pressure_patient_prescription_no",
        "blood_relative_heart_attack_before_age_50":"blood_relative_heart_attack_before_50_no",
        "urine_leakage_bother":"urine_leakage_bother_somewhat",
        "artificial_joints":"artificial_joints_yes",
        "hearing_difficulty_how_often":"hearing_difficulty_usually",
        "hearing_frustration_how_often":"hearing_frustration_usually",
        "wear_hearing_aid":"wear_hearing_aid_no",
        "loud_noise_job":"loud_noise_job_no",
        "loud_music":"loud_music_yes",
        "age_at_first_child":"age_at_first_child_20_through_24",
        "last_dentist_visit":"last_dentist_visit_7_months_to_a_year",
        "times_brush_teeth_daily":"no_times_brush_teeth_daily_1_or_less",
        "floss_days_per_week":"no_floss_days_per_week_3_or_less",
        "teeth_health":"no_teeth_health_poor",
        "gum_disease":"gum_disease_yes",
        "oral_cancer_exam":"oral_cancer_exam_no",
        "dental_bone_loss":"dental_bone_loss_no",
        "mouth_pain_last_year":"mouth_pain_last_year_fairly_often"
        
    }
)
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

Wall time: 5.38 s
start timing...
Explanations took 8.284611599999977 seconds


In [34]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'bmi': -0.03876620885937632, 'dietary_energy': -0.0387662199621861, 'daily_walk_or_bicycle_travel': -0.03876620885937632, 'daily_time_sitting': -0.038766208859376206, 'ten_minutes_daily_workout': -0.03876620885937632, 'marital_status': -0.03876620885937632, 'anxious_how_often': -0.03876620885937632, 'lack_of_exercise_reported': -0.277481321992888, 'obesity': -0.03876620885937632, 'liver_disorders': -0.03876620885937632, 'poor_oral_health': -0.03876620747152515, 'inflammation_from_illness': -0.03876620885937632, 'c_reactive_protein': -0.03876620885937632, 'poor_sleep_schedule': -0.03876620885937632, 'poor_sleep': -0.03876620747152515, 'inflammation_from_behavior': -0.1365315262334772, 'diet_fats': -0.03876620885937632, 'diet_basics': -0.03876620885937632, 'diet_quality_food': -0.03876620885937632, 'diet_quantity': -0.03876620885937632, 'diet_quality': -0.03876620885937632, 'inactivated_sirtuins': -0.03876620885937632, 'lung_or_kidney_or_liver_disease': -0.038766208

In [35]:
evidence1 = evidence.copy()
evidence1['diabetes']=  'diabetes'
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence1)

Wall time: 5.38 s


In [36]:
evidence1 = evidence.copy()
evidence1['a1c']=  'a1c_diabetes_5.7_and_above'
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence1)

Wall time: 5.35 s


In [37]:
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

start timing...
Explanations took 8.521957299999997 seconds


In [38]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'bmi': -0.037878728495637115, 'dietary_energy': -0.0378787395984469, 'daily_walk_or_bicycle_travel': -0.037878728495637115, 'daily_time_sitting': -0.037878728495637004, 'ten_minutes_daily_workout': -0.037878728495637115, 'marital_status': -0.037878728495637115, 'anxious_how_often': -0.037878728495637115, 'lack_of_exercise_reported': -0.2765938416291488, 'obesity': -0.037878728495637115, 'liver_disorders': -0.037878728495637115, 'poor_oral_health': -0.03787872710778595, 'inflammation_from_illness': -0.037878728495637115, 'c_reactive_protein': -0.037878728495637115, 'poor_sleep_schedule': -0.037878728495637115, 'poor_sleep': -0.03787872710778595, 'inflammation_from_behavior': -0.135644045869738, 'diet_vitamins_and_minerals': -0.037878728495637115, 'diet_fats': -0.037878728495637115, 'diet_basics': -0.037878728495637115, 'diet_quality_food': -0.037878728495637115, 'diet_quantity': -0.037878728495637115, 'diet_quality': -0.037878728495637115, 'inactivated_sirtuins': -

In [39]:
evidence1 = evidence.copy()
evidence1['age']=  'elderly'
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence1)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence1,outvars)

Wall time: 5.71 s


In [40]:
evidence ={}
evidence['age']= 'elderly'
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)

Wall time: 3.79 s


In [41]:
evidence ={}
evidence['lack_of_exercise_reported']= 'no_lack_of_exercise_reported'
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)

Wall time: 3.7 s


## Beta Tester Surveys

In [42]:
tester1_4 =  {"age": "elderly", "bmi": "bmi_under_25_healthy", "gender": "male", "physical_work": "physical_work_no", "fish_last_month": "fish_last_month_yes", "chore_difficulty": "chore_difficulty_some", "how_healthy_diet": "how_healthy_diet_excellent", "daily_time_sitting": "daily_time_sitting_12_or_more_hours", "walking_difficulty": "walking_difficulty_none", "ten_stairs_difficulty": "ten_stairs_difficulty_much", "water_drank_yesterday": "water_drank_yesterday_above_1633.50", "stand_from_sit_difficulty": "stand_from_sit_difficulty_much", "ten_minutes_daily_workout": "ten_minutes_daily_workout_no", "social_activity_difficulty": "social_activity_difficulty_some", "tuna_eaten_times_per_month": "tuna_eaten_per_month_8_or_more", "two_hour_sitting_difficulty": "two_hour_sitting_difficulty_none", "daily_walk_or_bicycle_travel": "daily_walk_or_bicycle_travel_less_than_30_minutes", "two_hour_standing_difficulty": "two_hour_standing_difficulty_some", "weekend_sleep_hours_per_night": "weekend_sleep_7.0_or_more", "workday_sleep_hours_per_night": "workday_sleep_under_5_no", "eat_out_how_many_times_per_week": "eat_out_1_or_2_times_per_week"}
evidence = tester1_4               
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)        
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

Wall time: 3.44 s
start timing...
Explanations took 7.5474377000000175 seconds


In [43]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'gender': 0.02499977858625857, 'daily_time_sitting': 0.024999771694471207, 'ten_minutes_daily_workout': 0.02499977858625857, 'daily_walk_or_bicycle_travel': 0.02499977858625857, 'eat_out_how_many_times_per_week': 0.02499977858625857, 'lack_of_exercise_reported': 0.2749997807269808, 'obesity': 0.02499977858625857, 'c_reactive_protein': 0.02499977858625857, 'diet_fats': 0.02499977858625857, 'diet_quality_food': 0.02499977858625857, 'hypertension': 0.02499977858625857, 'inactivated_sirtuins': 0.02499977858625857, 'chronic_conditions': 0.02499977858625857, 'comorbidities': 0.02499977858625857}, 'obesity': {'gender': 0.23816800937486526, 'daily_time_sitting': 0.23816801429705198, 'ten_minutes_daily_workout': 0.23816800937486526, 'daily_walk_or_bicycle_travel': 0.23816800937486526, 'eat_out_how_many_times_per_week': 0.23816800937486526, 'lack_of_exercise_reported': 0.23816800937486526, 'lack_of_exercise': 0.23816800937486526, 'c_reactive_protein': 0.23816800937486526, '

In [44]:
tester1_3= {"age": "elderly", "bmi": "bmi_under_25_healthy", "gender": "male", "physical_work": "physical_work_no", "chore_difficulty": "chore_difficulty_some", "daily_time_sitting": "daily_time_sitting_12_or_more_hours", "walking_difficulty": "walking_difficulty_none", "ten_stairs_difficulty": "ten_stairs_difficulty_much", "stand_from_sit_difficulty": "stand_from_sit_difficulty_much", "ten_minutes_daily_workout": "ten_minutes_daily_workout_no", "social_activity_difficulty": "social_activity_difficulty_some", "two_hour_sitting_difficulty": "two_hour_sitting_difficulty_none", "daily_walk_or_bicycle_travel": "daily_walk_or_bicycle_travel_less_than_30_minutes", "two_hour_standing_difficulty": "two_hour_standing_difficulty_some", "weekend_sleep_hours_per_night": "weekend_sleep_7.0_or_more", "workday_sleep_hours_per_night": "workday_sleep_under_5_no"} 
evidence = tester1_3
%time internal_query_result = internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)

Wall time: 3.71 s


In [45]:
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

start timing...
Explanations took 7.931436700000006 seconds


In [46]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'gender': 0.02499978087073851, 'daily_time_sitting': 0.024999773978951145, 'ten_minutes_daily_workout': 0.02499978087073851, 'daily_walk_or_bicycle_travel': 0.02499978087073851, 'lack_of_exercise_reported': 0.2749997830114607, 'obesity': 0.02499978087073851, 'c_reactive_protein': 0.02499978087073851, 'diet_fats': 0.02499978087073851, 'diet_basics': 0.02499978087073851, 'diet_quality_food': 0.02499978087073851, 'diet_quantity': 0.02499978087073851, 'diet_quality': 0.02499978087073851, 'hypertension': 0.02499978087073851, 'inactivated_sirtuins': 0.02499978087073851, 'chronic_conditions': 0.02499978087073851, 'comorbidities': 0.02499978087073851}, 'obesity': {'gender': 0.23816803735352743, 'daily_time_sitting': 0.23816804227571414, 'ten_minutes_daily_workout': 0.23816803735352743, 'daily_walk_or_bicycle_travel': 0.23816803735352743, 'lack_of_exercise_reported': 0.23816803735352743, 'lack_of_exercise': 0.23816803735352743, 'c_reactive_protein': 0.23816803735352743, 'd

In [47]:
tester1_2 = {"age": "elderly", "bmi": "bmi_under_25_healthy", "gender": "male", "physical_work": "physical_work_no", "chore_difficulty": "chore_difficulty_some", "daily_time_sitting": "daily_time_sitting_12_or_more_hours", "walking_difficulty": "walking_difficulty_none", "ten_stairs_difficulty": "ten_stairs_difficulty_much", "stand_from_sit_difficulty": "stand_from_sit_difficulty_much", "ten_minutes_daily_workout": "ten_minutes_daily_workout_no", "social_activity_difficulty": "social_activity_difficulty_some", "two_hour_sitting_difficulty": "two_hour_sitting_difficulty_none", "daily_walk_or_bicycle_travel": "daily_walk_or_bicycle_travel_less_than_30_minutes", "two_hour_standing_difficulty": "two_hour_standing_difficulty_some"}
evidence = tester1_2
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)

Wall time: 3.45 s


In [48]:
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

start timing...
Explanations took 7.708317299999976 seconds


In [49]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'gender': 0.024999765321727208, 'daily_time_sitting': 0.02499975815054345, 'ten_minutes_daily_workout': 0.024999765321727208, 'daily_walk_or_bicycle_travel': 0.024999765321727208, 'lack_of_exercise_reported': 0.2749997670084298, 'obesity': 0.024999765321727208, 'c_reactive_protein': 0.024999765321727208, 'diet_fats': 0.024999765321727208, 'diet_basics': 0.024999765321727208, 'diet_quality_food': 0.024999765321727208, 'diet_quantity': 0.024999765321727208, 'diet_quality': 0.024999765321727208, 'hypertension': 0.024999765321727208, 'inactivated_sirtuins': 0.024999765321727208, 'chronic_conditions': 0.024999765321727208, 'comorbidities': 0.024999765321727208}, 'obesity': {'gender': 0.22420973855848852, 'daily_time_sitting': 0.2242097410195819, 'ten_minutes_daily_workout': 0.22420973855848852, 'daily_walk_or_bicycle_travel': 0.22420973855848852, 'lack_of_exercise_reported': 0.22420973855848852, 'lack_of_exercise': 0.22420973855848852, 'c_reactive_protein': 0.224209738

In [50]:
tester1_1= {"age": "elderly", "bmi": "bmi_under_25_healthy", "gender": "male", "physical_work": "physical_work_no", "daily_time_sitting": "daily_time_sitting_12_or_more_hours", "ten_minutes_daily_workout": "ten_minutes_daily_workout_no", "daily_walk_or_bicycle_travel": "daily_walk_or_bicycle_travel_less_than_30_minutes"}
evidence = tester1_1 
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
internal_query_result
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)
# First try calling everyones result first, and sending it through for a check of the explanations one by one.
# you can call good and bad at the same time.
# Also check if it is indeed bad or good before you send it in one of the explanations
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

Wall time: 3.51 s
start timing...
Explanations took 7.710243100000014 seconds


In [51]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'gender': 0.024999767304389797, 'daily_time_sitting': 0.02499976013320593, 'ten_minutes_daily_workout': 0.024999767304389797, 'daily_walk_or_bicycle_travel': 0.024999767304389797, 'lack_of_exercise_reported': 0.2749997682926004, 'obesity': 0.024999767304389797, 'c_reactive_protein': 0.024999767304389797, 'diet_fats': 0.024999767304389797, 'diet_basics': 0.024999767304389797, 'diet_quality_food': 0.024999767304389797, 'diet_quantity': 0.024999767304389797, 'diet_quality': 0.024999767304389797, 'hypertension': 0.024999767304389797, 'inactivated_sirtuins': 0.024999767304389797, 'chronic_conditions': 0.024999767304389797, 'comorbidities': 0.024999767304389797}, 'obesity': {'gender': 0.22420973855300824, 'daily_time_sitting': 0.22420974101410163, 'ten_minutes_daily_workout': 0.22420973855300824, 'daily_walk_or_bicycle_travel': 0.22420973855300824, 'lack_of_exercise_reported': 0.22420973855300824, 'lack_of_exercise': 0.22420973855300824, 'c_reactive_protein': 0.22420973

In [52]:
tester1_5= {"age": "elderly", "gender": "male"}
evidence = tester1_5 
%time internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
internal_query_result
#we show some of the same results as in the last cell for clarity
outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations",
"hallmark_4_loss_of_proteostasis","hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction",
"hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion","hallmark_9_altered_intercellular_communication",
"hallmark_10_extracellular_matrix_dysfunction","poor_diet","poor_diet_flag","poor_sleep","psychological_disorders",
"obesity","lack_of_exercise","poor_diet_quantity","smoking"]
results = query(longevity,bayesianNetwork,evidence,outvars)
print("start timing...")
tic = time.perf_counter()
#internal_query_result =  internal_query(longevity,bayesianNetwork,evidence)
#for check_if_explains in include_list:
    #checklist = [check_if_explains]
checklist = include_list
why_bad = explain_why_bad(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
why_good = explain_why_good(longevity,bayesianNetwork,evidence,outvars,internal_query_result=internal_query_result, include_list = checklist)
toc = time.perf_counter()
diff = toc - tic
print (f"Explanations took {diff} seconds")

Wall time: 3.72 s
start timing...
Explanations took 7.1190348000000085 seconds


In [53]:
print(why_bad) 
print(why_good)

{'lack_of_exercise': {'gender': 0.2157746844507053, 'lack_of_exercise_reported': -0.03422531747464852, 'diet_fats': 0.2157746844507053, 'diet_basics': 0.2157746844507053, 'diet_quality_food': 0.2157746844507053, 'diet_quantity': 0.2157746844507053, 'diet_quality': 0.2157746844507053, 'inactivated_sirtuins': 0.2157746844507053, 'chronic_conditions': 0.2157746844507053}, 'obesity': {'gender': 0.19796870905154046, 'lack_of_exercise_reported': 0.19796870905154046, 'diet_fats': 0.19796870905154046, 'diet_basics': 0.19796870905154046, 'diet_quality_food': 0.19796870905154046, 'diet_quantity': 0.19796870905154046, 'diet_quality': 0.19796870905154046, 'inactivated_sirtuins': 0.19796870905154046, 'chronic_conditions': 0.19796871268624272}, 'smoking': {'gender': 0.23125000621814623, 'lack_of_exercise_reported': 0.2312500080807915, 'diet_fats': 0.2312500080807915, 'diet_basics': 0.2312500080807915, 'diet_quality_food': 0.2312500080807915, 'diet_quantity': 0.2312500080807915, 'diet_quality': 0.231

## Take a signal and put a fake anomaly at the end to test the sensitivity to wearables in the rules

We find that lack of exercise signal explains why lack of exersize is off track, even more so than obesity does. 

In [54]:
%run wearables_query.py

In [55]:
evidence

{'age': 'adult',
 'bmi': 'bmi_over_40_high_risk',
 'gender': 'female',
 'height': 'no_height_above_173.40',
 'weight': 'weight_above_92.60',
 'systolic': 'systolic_130_to_139_high_stage_1',
 'diastolic': 'diastolic_80_to_89_high_stage_1',
 'resting_heart_rate': 'no_resting_heart_rate_very_high_91_and_above'}

In [56]:
outvars

['hallmark_1_genomic_instability',
 'hallmark_2_telomere_attrition',
 'hallmark_3_epigenetic_alterations',
 'hallmark_4_loss_of_proteostasis',
 'hallmark_5_deregulated_nutrient_sensing',
 'hallmark_6_mitochondrial_dysfunction',
 'hallmark_7_cellular_senescence',
 'hallmark_8_stem_cell_exhaustion',
 'hallmark_9_altered_intercellular_communication',
 'hallmark_10_extracellular_matrix_dysfunction',
 'poor_diet',
 'poor_diet_flag',
 'poor_sleep',
 'psychological_disorders',
 'obesity',
 'lack_of_exercise',
 'poor_diet_quantity',
 'smoking']

In [57]:
include_list = ["dietary_energy", "bmi", "lack_of_exercise","anxious_how_often","daily_time_sitting","daily_walk_or_bicycle_travel","dietary_fiber","eat_out_how_many_times_per_week","five_days_smoke_cigarettes","gender","heart_rate_anomaly","marital_status","step_variability_anomaly","steps_anomaly","ten_minutes_daily_workout","workout_anomaly","blood_age_indicators","c_reactive_protein","cancer","cancer_related","cardiovascular_disease","chronic_conditions","comorbidities","diabetes","diet_quality","frailty","frailty_signals","frailty_signs","heart_rate_variability_anomaly","hypertension","inactivated_sirtuins","inflammation","inflammation_from_behavior","inflammation_from_diet","kidney_disease","lack_of_exercise","lung_or_kidney_or_liver_disease","metabolic_disease","oxygen_anomaly","poor_diet","poor_diet_flag","poor_oral_health","poor_sleep","psychological_disorders","sarcopenia","sleep_anomaly","smoking","uv_exposure","liver_disorders","heart_disorder_indicators","inflammation_from_illness","poor_diet_quantity","general_aging_signs","lack_of_exercise_reported","diet_basics","diet_fats","diet_quality_food","diet_quantity","diet_vitamins_and_minerals","blood_metabolism_disorder_indicators","obesity","poor_sleep_schedule"]
outvars.extend(include_list)

In [58]:
outvars.extend(["lack_of_exercise_signal","resting_heart_rate","signal_heart_disorders_indicators","steps_anomaly","workout_anomaly","heart_rate_anomaly","step_asymmetry_anomaly","step_variability_anomaly",
 "stride_variability_anomaly","heart_rate_variability_anomaly","walking_speed_variability_anomaly"])

In [59]:
data

{'steps_anomaly': {'data': [{'value': 4825.0,
    'record_date': '2022-03-23 23:29:20',
    'interval_in_second': 0},
   {'value': 7783.0,
    'record_date': '2022-03-24 23:53:48',
    'interval_in_second': 87868},
   {'value': 7594.0,
    'record_date': '2022-03-25 22:49:15',
    'interval_in_second': 82527},
   {'value': 1137.0,
    'record_date': '2022-03-26 22:55:13',
    'interval_in_second': 86758},
   {'value': 987.0,
    'record_date': '2022-03-27 23:42:13',
    'interval_in_second': 89220},
   {'value': 3667.0,
    'record_date': '2022-03-28 23:16:25',
    'interval_in_second': 84852},
   {'value': 6247.0,
    'record_date': '2022-03-29 23:06:16',
    'interval_in_second': 85791},
   {'value': 5142.0,
    'record_date': '2022-03-31 21:47:23',
    'interval_in_second': 168067},
   {'value': 4973.0,
    'record_date': '2022-04-01 22:37:50',
    'interval_in_second': 89427},
   {'value': 3428.0,
    'record_date': '2022-04-02 23:32:48',
    'interval_in_second': 89698},
   {'valu

In [60]:
if exists("bayesianNetwork.pkl"):
        infile = open("bayesianNetwork.pkl",'rb')
        bayesianNetwork = pickle.load(infile)
        infile.close()
else:
        %time bayesianNetwork,outstr = longevity_bayes.longevity_bayes()
        print (outstr)
        outfile = open("bayesianNetwork.pkl",'wb')
        pickle.dump(bayesianNetwork,outfile)
        outfile.close()

In [61]:
var_positions = get_var_positions(bayesianNetwork)
var_positions

{'fake_variable': 0,
 'age': 1,
 'daily_aspirin_compliance': 2,
 'high_blood_pressure_patient_prescription': 3,
 'high_blood_pressure_medication_compliance': 4,
 'diastolic': 5,
 'systolic': 6,
 'sun_exposure_half_hour': 7,
 'time_spent_outdoors': 8,
 'sun_exposure_use_sunscreen': 9,
 'sunburn_last_year': 10,
 'weight_at_25': 11,
 'bmi': 12,
 'dietary_energy': 13,
 'shortness_of_breath_exertion': 14,
 'daily_walk_or_bicycle_travel': 15,
 'daily_time_sitting': 16,
 'physical_work': 17,
 'ten_minutes_daily_workout': 18,
 'heart_rate_anomaly': 19,
 'steps_anomaly': 20,
 'workout_anomaly': 21,
 'walking_speed_anomaly': 22,
 'walking_speed_variability_anomaly': 23,
 'stride_variability_anomaly': 24,
 'step_asymmetry_anomaly': 25,
 'double_support_stepping_anomaly': 26,
 'step_variability_anomaly': 27,
 'cumin_supplementation': 28,
 'anxious_how_often': 29,
 'marital_status': 30,
 'birth_weight': 31,
 'asthma_attack_last_year': 32,
 'liver_condition': 33,
 'fatiqued_in_2_weeks': 34,
 'osteop

In [62]:
var_val_positions = get_var_val_positions(bayesianNetwork)
var_val_positions

{'fake_variable': {'fake_variable_1': 0,
  'fake_variable_2': 1,
  'fake_variable_3': 2,
  'fake_variable_4': 3,
  'fake_variable_5': 4},
 'age': {'elderly': 0, 'adult': 1, 'young_adult': 2, 'teen': 3, 'child': 4},
 'daily_aspirin_compliance': {'daily_aspirin_compliance_no': 0,
  'daily_aspirin_compliance_yes': 1},
 'high_blood_pressure_patient_prescription': {'high_blood_pressure_patient_prescription_yes': 0,
  'high_blood_pressure_patient_prescription_no': 1},
 'high_blood_pressure_medication_compliance': {'high_blood_pressure_medication_compliance_no': 0,
  'high_blood_pressure_medication_compliance_yes': 1},
 'diastolic': {'diastolic_120_or_higher_crisis': 0,
  'diastolic_90_to_119_high_stage_2': 1,
  'diastolic_80_to_89_high_stage_1': 2,
  'diastolic_less_than_80_normal': 3},
 'systolic': {'systolic_180_over_crisis': 0,
  'systolic_140_to_179_high_stage_2': 1,
  'systolic_130_to_139_high_stage_1': 2,
  'systolic_120_to_129_elevated': 3,
  'systolic_less_than_120_normal': 4},
 'sun

In [63]:
original_outvars = ["hallmark_1_genomic_instability","hallmark_2_telomere_attrition","hallmark_3_epigenetic_alterations","hallmark_4_loss_of_proteostasis",
        "hallmark_5_deregulated_nutrient_sensing","hallmark_6_mitochondrial_dysfunction","hallmark_7_cellular_senescence","hallmark_8_stem_cell_exhaustion",
        "hallmark_9_altered_intercellular_communication","hallmark_10_extracellular_matrix_dysfunction","poor_diet",
        "poor_diet_flag","poor_sleep","psychological_disorders","obesity","lack_of_exercise","poor_diet_quantity","smoking"]
explainvars=outvars
include_list.extend(["lack_of_exercise_signal","resting_heart_rate","signal_heart_disorders_indicators","steps_anomaly","workout_anomaly","heart_rate_anomaly","step_asymmetry_anomaly","step_variability_anomaly",
 "stride_variability_anomaly","heart_rate_variability_anomaly","walking_speed_variability_anomaly"])
query = Query()
query.switch = "internal_query"
for v in include_list:
    if v in var_positions:
        outvar = query.include_list.add()
        outvar.var_num = var_positions[v]
for var,val  in  evidence.items():
    e= query.evidence.add()
    e.var_num = var_positions[var] if var in var_positions else 0
    e.response = var_val_positions[var][val] if var in var_positions and val in var_val_positions[var] else 0
for o in outvars:
    outvar = query.outvars.add()
    outvar.var_num = var_positions[o] if o in var_positions else 0
for v in explainvars:
    if v in var_positions:
        explainvar = query.explainvars.add()
        explainvar.var_num = var_positions[v]
for var,data_dict in data.items():
    print("var")
    print(var)
    timeseries = query.timeseries.add()
    timeseries.var_num = var_positions[var] if var in var_positions else 0
    for r in data_dict["data"]:
        time = timeseries.timevals.add()
        time.val = r["value"] if "value" in r else 0
        time.interval = r["interval_in_second"] if "interval_in_second" in r else 0
        last = time.interval
    # put anomaly in last timeframe    
    time = timeseries.timevals.add()
    time.interval = last
    if var == "steps_anomaly":
        time.val = 0
    elif var == "workout_anomaly":
        time.val = 0
    elif var == "heart_rate_anomaly":
        time.val = 125
    elif var == "step_asymmetry_anomaly":
        time.val = 0.99  
    elif var == "step_variability_anomaly":
        time.val = 1000000 #you cant really make variability change just by changing the last number
    elif var == "stride_variability_anomaly":
        time.val = 20.0
    elif var == "heart_rate_variability_anomaly":
        time.val = 125 # correct to hrv val when fixed
    elif var == "walking_speed_variability_anomaly":
        time.val = 50.0

var
steps_anomaly
var
workout_anomaly
var
heart_rate_anomaly
var
step_asymmetry_anomaly
var
step_variability_anomaly
var
stride_variability_anomaly
var
heart_rate_variability_anomaly
var
walking_speed_variability_anomaly


In [64]:
query

evidence {
  var_num: 1
  response: 1
}
evidence {
  var_num: 12
}
evidence {
  var_num: 104
  response: 1
}
evidence {
  var_num: 168
  response: 1
}
evidence {
  var_num: 167
}
evidence {
  var_num: 6
  response: 2
}
evidence {
  var_num: 5
  response: 2
}
evidence {
  var_num: 204
  response: 1
}
outvars {
  var_num: 288
}
outvars {
  var_num: 289
}
outvars {
  var_num: 302
}
outvars {
  var_num: 307
}
outvars {
  var_num: 309
}
outvars {
  var_num: 293
}
outvars {
  var_num: 305
}
outvars {
  var_num: 310
}
outvars {
  var_num: 311
}
outvars {
  var_num: 312
}
outvars {
  var_num: 268
}
outvars {
  var_num: 251
}
outvars {
  var_num: 228
}
outvars {
  var_num: 274
}
outvars {
  var_num: 177
}
outvars {
  var_num: 174
}
outvars {
  var_num: 249
}
outvars {
  var_num: 189
}
outvars {
  var_num: 13
}
outvars {
  var_num: 12
}
outvars {
  var_num: 174
}
outvars {
  var_num: 29
}
outvars {
  var_num: 16
}
outvars {
  var_num: 15
}
outvars {
  var_num: 120
}
outvars {
  var_num: 109
}
ou